In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from pydocumentdb import document_client
from scipy.stats import spearmanr

%matplotlib inline

# Data loading

Define Cosmos DB credentials.

In [ ]:
COSMOSDB_ENDPOINT = '<Your Cosmos DB endpoint>'
COSMOSDB_KEY = '<Your Cosmos DB "primaryMasterKey">'

Create a Cosmos DB client, select our database and collection, and define a function to run queries on the latter.

In [ ]:
client = document_client.DocumentClient(COSMOSDB_ENDPOINT, {'masterKey': COSMOSDB_KEY})
db = next(x for x in client.ReadDatabases() if x['id'] == 'ted')
coll = next(x for x in client.ReadCollections(db['_self']) if x['id'] == 'talks')
query_talks = lambda q: list(client.QueryDocuments(coll['_self'], {'query': q}))

Select the most recently published talk.

**Note**: Cosmos DB uses its own [SQL dialect](https://docs.microsoft.com/en-us/azure/cosmos-db/sql-api-sql-query), which you can test out using the [Query Playground](https://www.documentdb.com/sql/demo).

In [ ]:
query_talks('SELECT TOP 1 * FROM talks t ORDER BY t.published_at DESC')

Extract all the talks and create a `DataFrame`.

In [ ]:
all_talks = query_talks('SELECT * FROM talks t')
talks_df = pd.DataFrame(all_talks)

In [ ]:
talks_df.head()

# Data cleaning

Create a new variable `scraped_at` from the Cosmos DB `_ts` attribute.

In [ ]:
talks_df['scraped_at'] = pd.to_datetime(talks_df['_ts'], unit='s')

Get rid of Cosmos DB attributes.

In [ ]:
talks_df.drop(columns=['_attachments', '_etag', '_rid', '_self', '_ts'], inplace=True)

Fix data types for `filmed_at` and `published_at`.

In [ ]:
talks_df['filmed_at'] = pd.to_datetime(talks_df['filmed_at'], unit='s')
talks_df['published_at'] = pd.to_datetime(talks_df['published_at'], unit='s')

Convert `duration` to minutes.

In [ ]:
talks_df['duration'] /= 60

Check sanity of values.

In [ ]:
talks_df.describe(exclude='object')

In [ ]:
talks_df.sort_values('filmed_at')

Create a few more variables.

In [ ]:
talks_df['external_links_count'] = talks_df['external_links'].apply(len)
talks_df['speakers_count'] = talks_df['speakers'].apply(len)
talks_df['tags_count'] = talks_df['tags'].apply(len)
talks_df['publishing_delay'] = (talks_df['published_at'] - talks_df['filmed_at']) / np.timedelta64(1, 'D')
talks_df['total_ratings'] = talks_df['ratings'].apply(lambda x: sum(x.values()))
talks_df['views_per_day'] = talks_df['viewed'] / \
                            ((talks_df['scraped_at'] - talks_df['published_at']) / np.timedelta64(1, 'D'))

Extract `ratings` into separate variables.

In [ ]:
rating_names = {name for names in talks_df['ratings'] for name in names}

In [ ]:
rating_names

In [ ]:
for rating_name in rating_names:
    talks_df['rated_' + rating_name.lower()] = talks_df['ratings'].apply(lambda x: x.get(rating_name, 0)) / \
                                               talks_df['total_ratings']

Create dummy variables for the most common `tags` (present in ≥ 5% of the talks).

In [ ]:
all_tags = Counter(tag for tags in talks_df['tags'] for tag in tags)
frequent_tags = {tag for tag, count in all_tags.items() if count >= 0.05 * len(talks_df)}

In [ ]:
frequent_tags

In [ ]:
for tag in frequent_tags:
    talks_df['tagged_' + tag.lower().replace(' ', '_')] = talks_df['tags'].apply(lambda x: tag in x).astype(int)

In [ ]:
talks_df.head()

In [ ]:
talks_df.dtypes

# Exploratory data analysis

## Distribution of talk duration

In [ ]:
sns.distplot(talks_df['duration'], color='darkblue')  # In minutes

## Top 20 events by number of talks

In [ ]:
talks_df['event'].value_counts().head(20).plot.bar(color='darkblue')

## Number of external links

**Note**: the rightmost bar is for 10+ links.

In [ ]:
pd.cut(talks_df['external_links_count'],
       bins=list(range(11)) + [np.inf],
       labels=False,
       include_lowest=True).value_counts().sort_index().plot.bar(color='darkblue')

## Talks filmed by year

In [ ]:
talks_df.set_index('filmed_at') \
    .resample('Y')['id'] \
    .count() \
    .plot.line(color='darkblue')

Excluding very old talks.

In [ ]:
talks_df[talks_df['filmed_at'] >= '2000'] \
    .set_index('filmed_at') \
    .resample('Y')['id'].count() \
    .plot.line(color='darkblue')

## Distribution of publishing delay

In [ ]:
sns.distplot(talks_df['publishing_delay'], color='darkblue')

Excluding very old talks.

In [ ]:
sns.distplot(talks_df[talks_df['filmed_at'] >= '2000']['publishing_delay'], color='darkblue')

## Number of ratings

In [ ]:
talks_df['total_ratings'].describe()

In [ ]:
sns.distplot(np.log10(talks_df['total_ratings']), color='darkblue')

## Number of speakers

In [ ]:
talks_df['speakers_count'].value_counts().sort_index().plot.bar(color='darkblue')

## Number of tags

**Note**: the rightmost bar is for 20+ tags.

In [ ]:
pd.cut(talks_df['tags_count'],
       bins=list(range(21)) + [np.inf],
       labels=False,
       include_lowest=True).value_counts().sort_index().plot.bar(color='darkblue')

## Distribution of views and views/day

In [ ]:
sns.distplot(np.log10(talks_df['viewed']), color='darkblue')

In [ ]:
sns.distplot(np.log10(talks_df['views_per_day']), color='darkblue')

## Spearman correlations with views/day

In [ ]:
columns = talks_df.select_dtypes(exclude=['datetime', 'object']).columns
rating_corr = pd.DataFrame({
    'variable': columns,
    'r': [spearmanr(talks_df[x], talks_df['views_per_day']).correlation for x in columns],
}).sort_values('r')

In [ ]:
rating_corr[np.abs(rating_corr['r']) > 0.1]